# Vet a Known Exoplanet
This notebook pulls the parameters and light curve of a known exoplanet, WASP-18 b. It then runs three vetters on on that TCE and light curve. It is used as an example of how to run the vetters in the exovetter package.

In [ ]:
import numpy as np
import requests
import matplotlib.pyplot as plt
import pandas as p
from IPython.display import display, HTML

from astropy import units as u
import lightkurve as lk
from exovetter import const as exo_const
from exovetter.model import create_box_model_for_tce
from exovetter.tce import Tce
from exovetter import vetters as vet

In [ ]:
%matplotlib inline

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
header={}

## Create a TCE using exo.MAST requests.
Since we are vetting a known exoplanet, we pull down the information about the exoplanet transit from exo.mast's API. We then create a TCE for vetting. A TCE is an object that contains the transit ephemeris and basic parameters that usually come out of a transit search pipeline (period, epoch, snr, depth).

In [ ]:
planet_name = "WASP-18 b"

url = planeturl + "/identifiers/"
myparams = {"name":planet_name}
r = requests.get(url = url, params = myparams, headers = header)
planet_names = r.json()
ticid = planet_names['tessID']
tceid = planet_names['tessTCE']

In [ ]:
url = planeturl + planet_name + "/properties/"

r = requests.get(url = url, headers = header)

planet_prop = r.json()
print("Catalog 2: " + planet_prop[1]['catalog_name'])
planet_prop[0].keys()
nexsci_prop = planet_prop[1]  #Second one returned

In [ ]:
tce = Tce(period = nexsci_prop['orbital_period'] * u.day,
         epoch = nexsci_prop['transit_time'] * u.day,
         epoch_offset =  -2_400_000.5 * u.day,
         depth = nexsci_prop['transit_depth'] * exo_const.frac_amp,
         duration = nexsci_prop['transit_duration'] * u.day,
         snr = 50, #This value is only used by LPP and doesn't need to be very precise
         event_name = "WASP-18 b",
         target_name = "WASP-18")

# Get a lightkurve for this object
We use the lightkurve package to pull down sector 3 data for TESS data on this object. We then flatten that data to remove any remaining instrumental systematics.

In [ ]:
lcf = lk.search_lightcurvefile(tce['target_name'], mission='TESS', sector = 3).download()
lc = lcf.SAP_FLUX.remove_nans().remove_outliers()
flat = lc.flatten(window_length=81)

In [ ]:
flat.plot()

In [ ]:
lpp = vet.Lpp(lc_name="flux")
_ = lpp.run(tce,flat)

In [ ]:
lpp.plot()

In [ ]:
oe = vet.OddEven()
_ = oe.run(tce,flat)
oe.plot()

In [ ]:
tcover = vet.TransitPhaseCoverage()
_ = tcover.run(tce,flat)
tcover.plot()

In [ ]:
tcover.lc_name

In [ ]:
#Each Vetter can be returned as a dictionary
tcover.__dict__

In [ ]:
#Exovetter uses the lightcurve time_format for units on the times.
lc.time_format